## Step 1: Load and Inspect Data

In [ ]:

import pandas as pd

# Load Excel file
file_path = "Customer_Churn_Data_Large.xlsx"
excel_file = pd.ExcelFile(file_path)

# Load sheets
demographics_df = excel_file.parse('Customer_Demographics')
transactions_df = excel_file.parse('Transaction_History')
service_df = excel_file.parse('Customer_Service')
activity_df = excel_file.parse('Online_Activity')
churn_df = excel_file.parse('Churn_Status')

# Preview
print(demographics_df.head())
print(transactions_df.head())


## Step 2: Merge All Datasets

In [ ]:

merged_df = churn_df.merge(demographics_df, on='CustomerID', how='left')
merged_df = merged_df.merge(activity_df, on='CustomerID', how='left')
merged_df = merged_df.merge(service_df, on='CustomerID', how='left')
merged_df = merged_df.merge(transactions_df, on='CustomerID', how='left')

# Convert date columns
merged_df['TransactionDate'] = pd.to_datetime(merged_df['TransactionDate'])
merged_df['InteractionDate'] = pd.to_datetime(merged_df['InteractionDate'])
merged_df['LastLoginDate'] = pd.to_datetime(merged_df['LastLoginDate'])
merged_df.info()


## Step 3: Data Cleaning

In [ ]:

merged_df['InteractionType'] = merged_df['InteractionType'].fillna('No Interaction')
merged_df['ResolutionStatus'] = merged_df['ResolutionStatus'].fillna('None')
merged_df['InteractionID'] = merged_df['InteractionID'].fillna(0)
merged_df['InteractionDate'] = merged_df['InteractionDate'].fillna(pd.Timestamp('1900-01-01'))


## Step 4: Encoding and Normalization

In [ ]:

from sklearn.preprocessing import StandardScaler

# Encode categorical
encoded_df = pd.get_dummies(merged_df, columns=[
    'Gender', 'MaritalStatus', 'IncomeLevel', 'ServiceUsage',
    'InteractionType', 'ResolutionStatus', 'ProductCategory'
], drop_first=True)

# Normalize numeric
scaler = StandardScaler()
encoded_df[['Age', 'LoginFrequency', 'AmountSpent']] = scaler.fit_transform(
    encoded_df[['Age', 'LoginFrequency', 'AmountSpent']]
)

# Drop unused columns
cleaned_df = encoded_df.drop(columns=['CustomerID', 'TransactionID', 'TransactionDate', 'InteractionID', 'InteractionDate', 'LastLoginDate'])
cleaned_df.head()


## Step 5: Exploratory Data Analysis

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

# Churn count
plt.figure(figsize=(6, 4))
sns.countplot(data=cleaned_df, x='ChurnStatus')
plt.title('Churn Distribution')
plt.xlabel('Churn Status (0 = Active, 1 = Churned)')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

# Box plots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
sns.boxplot(data=cleaned_df, x='ChurnStatus', y='Age', ax=axes[0])
sns.boxplot(data=cleaned_df, x='ChurnStatus', y='LoginFrequency', ax=axes[1])
sns.boxplot(data=cleaned_df, x='ChurnStatus', y='AmountSpent', ax=axes[2])
axes[0].set_title('Age vs Churn')
axes[1].set_title('Login Frequency vs Churn')
axes[2].set_title('Amount Spent vs Churn')
plt.tight_layout()
plt.show()

# Correlation heatmap
plt.figure(figsize=(12, 8))
correlation = cleaned_df.corr()
sns.heatmap(correlation, cmap='coolwarm', annot=True, fmt='.2f', center=0)
plt.title('Feature Correlation Heatmap')
plt.tight_layout()
plt.show()
